In [4]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict
from langgraph.types import Command, interrupt

mem = MemorySaver()
class State(TypedDict):
    text: str

def node_a(state:State):
    print("Node A")
    return Command(
        goto="node_b",
        update={
            "text": state["text"] + "a"
        }
    )

def node_b(state:State):
    print("Node B")
    human_interrupt = interrupt("should I go to c/d? c/d")
    print(human_interrupt, 'interrupt')
    if(human_interrupt == 'c'):
        return Command(
        goto="node_c",
        update={
            "text": state["text"] + "b"
        }
    )
    elif human_interrupt == 'd':
        return Command(
        goto="node_d",
        update={
            "text": state["text"] + "b"
        }
    )

    

def node_c(state:State):
    print("Node c")
    return Command(
        goto="node_d",
        update={
            "text":state["text"] + "c"
        }
    )

def node_d(state:State):
    print("Node D")
    return Command(
        goto=END,
        update={
            "text": state["text"] + "d"
        }
    )

graph = StateGraph(State)
graph.add_node('node_a', node_a)
graph.add_node('node_b', node_b)
graph.add_node('node c', node_c)
graph.add_node('node_d', node_d)
graph.set_entry_point('node_a')
app = graph.compile(checkpointer=mem)
print(app.get_graph().draw_mermaid())
config = {"configurable": {"thread_id": "1"}}

result = app.invoke({
    "text": "",
}, config=config, stream_mode="updates")

print(result)



---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__(<p>__start__</p>)
	node_a(node_a)
	node_b(node_b)
	node_c(node c)
	node_d(node_d)
	__end__(<p>__end__</p>)
	__start__ --> node_a;
	node_a --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

Node A
Node B
[{'node_a': {'text': 'a'}}, {'__interrupt__': (Interrupt(value='should I go to c/d? c/d', resumable=True, ns=['node_b:9e3d261c-8fcd-c7ca-0356-ea75dd182d37']),)}]


In [5]:
print(app.get_state(config).next)

('node_b',)


In [6]:
second_result = app.invoke(Command(resume="C"), config=config, stream_mode="updates")
second_result

Node B
C interrupt


[{'node_b': None}]